In [1]:
import sqlite3
import pandas as pd
con = sqlite3.connect("C:/Users/ams-duongn.EXPEDITORS/Desktop/learn/Data Science Master/M4W3/chinook.db")

In [2]:
# 1. How many customers are there that are not residing in the US?
number_non_us_customers = pd.read_sql_query('SELECT count(distinct c.CustomerId) as Number_Non_US_Customers\
                            FROM customers as c\
                            WHERE 1 = (case when c.Country = "USA" then 0 else 1 end)', con)
number_non_us_customers

,Number_Non_US_Customers
0,46


In [3]:
# 2. Show all customers that are from Brazil. How Brazilian customers are there?
brazilian_customers = pd.read_sql_query('SELECT c.*\
                            FROM customers as c\
                            WHERE 1 = (case when c.Country = "Brazil" then 1 else 0 end)', con)
brazilian_customers

,CustomerId,FirstName,LastName,Company,Address,City,State,Country,PostalCode,Phone,Fax,Email,SupportRepId
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,10,Eduardo,Martins,Woodstock Discos,"Rua Dr. Falcão Filho, 155",São Paulo,SP,Brazil,01007-010,+55 (11) 3033-5446,+55 (11) 3033-4564,eduardo@woodstock.com.br,4
2,11,Alexandre,Rocha,Banco do Brasil S.A.,"Av. Paulista, 2022",São Paulo,SP,Brazil,01310-200,+55 (11) 3055-3278,+55 (11) 3055-8131,alero@uol.com.br,5
3,12,Roberto,Almeida,Riotur,"Praça Pio X, 119",Rio de Janeiro,RJ,Brazil,20040-020,+55 (21) 2271-7000,+55 (21) 2271-7070,roberto.almeida@riotur.gov.br,3
4,13,Fernanda,Ramos,None,Qe 7 Bloco G,Brasília,DF,Brazil,71020-677,+55 (61) 3363-5547,+55 (61) 3363-7855,fernadaramos4@uol.com.br,4


In [4]:
number_brazilian_customers = brazilian_customers[["CustomerId"]].count(axis='index')
print(f'There are {number_brazilian_customers[0]} Brazilian customers in this dataset.')

There are 5 Brazilian customers in this dataset.


In [5]:
# 3. Show Invoices of customers who are from the US. The resultant table should show the customer's full name, Invoice ID, Date of the invoice and billing country.
invoice_data = pd.read_sql_query('SELECT (c.FirstName || " " || c.LastName) as Full_Customer_Name,\
                            i.InvoiceId, Date(i.InvoiceDate) AS "Invoice Date", i.BillingCountry\
                            FROM customers as c\
                            INNER JOIN invoices as i on c.CustomerId = i.CustomerId\
                            WHERE 1 = (case when c.Country = "USA" then 1 else 0 end)', con)
invoice_data

,Full_Customer_Name,InvoiceId,Invoice Date,BillingCountry
0,Frank Harris,13,2009-02-19,USA
1,Frank Harris,134,2010-08-13,USA
2,Frank Harris,145,2010-09-23,USA
3,Frank Harris,200,2011-05-24,USA
4,Frank Harris,329,2012-12-28,USA
...,...,...,...,...
86,Julia Barnett,137,2010-08-18,USA
87,Julia Barnett,266,2012-03-24,USA
88,Julia Barnett,289,2012-06-26,USA
89,Julia Barnett,311,2012-09-28,USA


In [6]:
#4. How many Employees are working as Sales Agents?
number_sales_agent = pd.read_sql_query('SELECT count(distinct e.EmployeeId) as Number_Sales_Agent\
                            FROM employees as e\
                            WHERE 1 = (case when e.Title = "Sales Support Agent" then 1 else 0 end)', con)
print(f'There are {number_sales_agent.iloc[:, 0][0]} employees working as Sales Agents in this dataset.')

There are 3 employees working as Sales Agents in this dataset.


In [7]:
#5. To which countries the invoices were sent? Show the list of all countries.
billing_countries = pd.read_sql_query('SELECT DISTINCT i.BillingCountry\
                            FROM invoices as i\
                            ORDER BY i.BillingCountry', con)
billing_countries

,BillingCountry
0,Argentina
1,Australia
2,Austria
3,Belgium
4,Brazil
5,Canada
6,Chile
7,Czech Republic
8,Denmark
9,Finland


In [8]:
#6. How many Invoices were there in 2010 and 2011? What are the respective total sales for each of those years?
invoice_data_per_year = pd.read_sql_query('SELECT A.Invoice_Year, count(distinct A.InvoiceId) as Number_invoices_per_year, sum(A.Total) as Total_sales_per_year\
                            FROM (\
                            SELECT DISTINCT i.*, strftime("%Y",i.InvoiceDate) as Invoice_Year\
                            FROM invoices as i) AS A\
                            WHERE CAST(A.Invoice_Year AS INT) BETWEEN 2010 AND 2011\
                            GROUP BY A.Invoice_Year', con)
invoice_data_per_year

,Invoice_Year,Number_invoices_per_year,Total_sales_per_year
0,2010,83,481.45
1,2011,83,469.58


In [9]:
#7. How many Invoices were created for each country?
invoice_data_per_country = pd.read_sql_query('SELECT i.BillingCountry, count(distinct i.InvoiceId) as Number_invoices_per_country, sum(i.Total) as Total_sales_per_country\
                            FROM invoices as i\
                            GROUP BY i.BillingCountry', con)
invoice_data_per_country

,BillingCountry,Number_invoices_per_country,Total_sales_per_country
0,Argentina,7,37.62
1,Australia,7,37.62
2,Austria,7,42.62
3,Belgium,7,37.62
4,Brazil,35,190.10
5,Canada,56,303.96
6,Chile,7,46.62
7,Czech Republic,14,90.24
8,Denmark,7,37.62
9,Finland,7,41.62


In [10]:
#8. How many tracks are there in each playlist?
tracks_per_playlist = pd.read_sql_query('SELECT p.Name, count(distinct t.TrackId) as Number_Tracks_per_Playlist\
                            FROM playlists as p\
                            INNER JOIN playlist_track as pt using (PlaylistId)\
                            INNER JOIN tracks as t using (TrackId)\
                            GROUP BY p.Name', con)
tracks_per_playlist

,Name,Number_Tracks_per_Playlist
0,90’s Music,1477
1,Brazilian Music,39
2,Classical,75
3,Classical 101 - Deep Cuts,25
4,Classical 101 - Next Steps,25
5,Classical 101 - The Basics,25
6,Grunge,15
7,Heavy Metal Classic,26
8,Music,3290
9,Music Videos,1


In [11]:
#9. How much in total did every Sales Agent make?
sales_per_agent = pd.read_sql_query('SELECT (e.FirstName || " " || e.LastName) AS Sales_Agent_Name, sum(i.Total) as Total_Sale\
                            FROM employees as e\
                            INNER JOIN customers as c on c.SupportRepId = e.EmployeeId\
                            INNER JOIN invoices as i using(CustomerId)\
                            GROUP BY (e.FirstName || " " || e.LastName)', con)
sales_per_agent

,Sales_Agent_Name,Total_Sale
0,Jane Peacock,833.04
1,Margaret Park,775.40
2,Steve Johnson,720.16


In [12]:
#10. What is the sales agent that made the most sales in 2012?
top_sales_agent_2012 = pd.read_sql_query('WITH TOP_SALES AS (SELECT (e.FirstName || " " || e.LastName) AS Sales_Agent_Name, sum(i.Total) as Total_Sale\
                            FROM employees as e\
                            INNER JOIN customers as c on c.SupportRepId = e.EmployeeId\
                            INNER JOIN invoices as i using(CustomerId)\
                            WHERE CAST((strftime("%Y",i.InvoiceDate)) AS INT) = 2012\
                            GROUP BY (e.FirstName || " " || e.LastName))\
                            SELECT A.Sales_Agent_Name, A.Total_Sale\
                            FROM (\
                            SELECT TOP_SALES.*, ROW_NUMBER() OVER (ORDER BY TOP_SALES.Total_Sale DESC) AS SEQ\
                            FROM TOP_SALES) A\
                            WHERE A.SEQ = 1', con)
top_sales_agent_2012

,Sales_Agent_Name,Total_Sale
0,Margaret Park,197.2


In [13]:
#11. Which sales agent made the most in sales overall?
top_sales_agent_overall = pd.read_sql_query('WITH TOP_SALES AS (SELECT (e.FirstName || " " || e.LastName) AS Sales_Agent_Name, sum(i.Total) as Total_Sale\
                            FROM employees as e\
                            INNER JOIN customers as c on c.SupportRepId = e.EmployeeId\
                            INNER JOIN invoices as i using(CustomerId)\
                            GROUP BY (e.FirstName || " " || e.LastName))\
                            SELECT A.Sales_Agent_Name, A.Total_Sale\
                            FROM (\
                            SELECT TOP_SALES.*, ROW_NUMBER() OVER (ORDER BY TOP_SALES.Total_Sale DESC) AS SEQ\
                            FROM TOP_SALES) A\
                            WHERE A.SEQ = 1', con)
top_sales_agent_overall

,Sales_Agent_Name,Total_Sale
0,Jane Peacock,833.04


In [14]:
#12. What is the number of customers assigned to each sales agent?
number_customers_per_agent = pd.read_sql_query('SELECT (e.FirstName || " " || e.LastName) AS Sales_Agent_Name, count(distinct c.CustomerId) as Number_Customers_Assigned_per_agent\
                            FROM employees as e\
                            INNER JOIN customers as c on c.SupportRepId = e.EmployeeId\
                            GROUP BY (e.FirstName || " " || e.LastName)', con)
number_customers_per_agent

,Sales_Agent_Name,Number_Customers_Assigned_per_agent
0,Jane Peacock,21
1,Margaret Park,20
2,Steve Johnson,18


In [15]:
#13. Which country's customers spent the most?
top_customers_country_spent_overall = pd.read_sql_query('WITH TOP_SALES AS (SELECT c.Country AS Customers_Country, sum(i.Total) as Total_Spend\
                            FROM customers as c\
                            INNER JOIN invoices as i using(CustomerId)\
                            GROUP BY c.Country)\
                            SELECT A.Customers_Country, A.Total_Spend\
                            FROM (\
                            SELECT TOP_SALES.*, ROW_NUMBER() OVER (ORDER BY TOP_SALES.Total_Spend DESC) AS SEQ\
                            FROM TOP_SALES) A\
                            WHERE A.SEQ = 1', con)
top_customers_country_spent_overall

,Customers_Country,Total_Spend
0,USA,523.06


In [16]:
#14. Show the most purchased track of 2010.
top_purchased_track_2010 = pd.read_sql_query('WITH TOP_SALES AS (SELECT t.Name AS Track_Name, sum(i.Total) as Total_Sale\
                            FROM tracks as t\
                            INNER JOIN invoice_items as ii using (TrackId)\
                            INNER JOIN invoices as i using(InvoiceId)\
                            WHERE CAST((strftime("%Y",i.InvoiceDate)) AS INT) = 2010\
                            GROUP BY t.Name)\
                            SELECT A.Track_Name, A.Total_Sale\
                            FROM (\
                            SELECT TOP_SALES.*, ROW_NUMBER() OVER (ORDER BY TOP_SALES.Total_Sale DESC) AS SEQ\
                            FROM TOP_SALES) A\
                            WHERE A.SEQ = 1', con)
top_purchased_track_2010

,Track_Name,Total_Sale
0,Dezesseis,22.77


In [17]:
#15. Show the top 5 most purchased tracks ever.
top_5_purchased_track = pd.read_sql_query('WITH TOP_SALES AS (SELECT t.Name AS Track_Name, sum(i.Total) as Total_Sale\
                            FROM tracks as t\
                            INNER JOIN invoice_items as ii using (TrackId)\
                            INNER JOIN invoices as i using(InvoiceId)\
                            GROUP BY t.Name)\
                            SELECT A.Track_Name, A.Total_Sale\
                            FROM (\
                            SELECT TOP_SALES.*, ROW_NUMBER() OVER (ORDER BY TOP_SALES.Total_Sale DESC) AS SEQ\
                            FROM TOP_SALES) A\
                            WHERE A.SEQ <= 5', con)
top_5_purchased_track

,Track_Name,Total_Sale
0,The Trooper,49.50
1,Eruption,45.54
2,The Fix,43.77
3,The Woman King,43.77
4,Walkabout,43.77


In [18]:
#16. What are the top 3 best selling artists?
top_3_best_selling_artists = pd.read_sql_query('WITH TOP_SALES AS (SELECT at.Name AS Artist_Name, sum(i.Total) as Total_Sale\
                            FROM artists as at\
                            INNER JOIN albums a using (ArtistId)\
                            INNER JOIN tracks as t using (AlbumId)\
                            INNER JOIN invoice_items as ii using (TrackId)\
                            INNER JOIN invoices as i using(InvoiceId)\
                            GROUP BY at.Name)\
                            SELECT A.Artist_Name, A.Total_Sale\
                            FROM (\
                            SELECT TOP_SALES.*, ROW_NUMBER() OVER (ORDER BY TOP_SALES.Total_Sale DESC) AS SEQ\
                            FROM TOP_SALES) A\
                            WHERE A.SEQ <= 3', con)
top_3_best_selling_artists

,Artist_Name,Total_Sale
0,Iron Maiden,1233.54
1,U2,895.59
2,Lost,833.70


In [19]:
#17. What is the most purchased Media Type?
top_purchased_media_type = pd.read_sql_query('WITH TOP_SALES AS (SELECT md.Name AS Media_Type, sum(i.Total) as Total_Sale\
                            FROM media_types md\
                            INNER JOIN tracks as t using (MediaTypeId)\
                            INNER JOIN invoice_items as ii using (TrackId)\
                            INNER JOIN invoices as i using(InvoiceId)\
                            GROUP BY md.Name)\
                            SELECT A.Media_Type, A.Total_Sale\
                            FROM (\
                            SELECT TOP_SALES.*, ROW_NUMBER() OVER (ORDER BY TOP_SALES.Total_Sale DESC) AS SEQ\
                            FROM TOP_SALES) A\
                            WHERE A.SEQ = 1', con)
top_purchased_media_type

,Media_Type,Total_Sale
0,MPEG audio file,17838.27


In [20]:
#18. Show the number of tracks purchased in all invoices that contain more than one genre.
multi_genres_tracks = pd.read_sql_query('WITH MULTI_GENRE AS (SELECT t.Name AS Track_Name, GROUP_CONCAT(g.Name,"; ") AS genres_list, count(distinct g.GenreId) as Genres_Count\
                            FROM genres g\
                            INNER JOIN tracks as t using (GenreId)\
                            INNER JOIN invoice_items as ii using (TrackId)\
                            INNER JOIN invoices as i using(InvoiceId)\
                            GROUP BY t.Name)\
                            SELECT MULTI_GENRE.Track_Name, MULTI_GENRE.genres_list\
                            FROM MULTI_GENRE\
                            WHERE MULTI_GENRE.Genres_Count > 1', con)
multi_genres_tracks

,Track_Name,genres_list
0,Before You Accuse Me,Rock; Blues
1,Believe,Jazz; Rock
2,Blood Brothers,Rock; Metal; Metal
3,Branch Closing,Comedy; TV Shows
4,Drifter,Rock; Heavy Metal
5,Flying High Again,Rock; Rock; Metal
6,Ghost Of The Navigator,Rock; Metal
7,Good Golly Miss Molly,Rock; Rock And Roll; Rock
8,Hallowed Be Thy Name,Metal; Rock; Rock; Rock
9,Heaven Can Wait,Metal; Rock; Metal
